In [1]:
import time
import akshare as ak
import pandas as pd
import numpy as np
import tqdm
# import pyecharts.options as opts
# from pyecharts.charts import Line
import pandas as pd
from sqlalchemy import create_engine
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

C:\Users\cyw\AppData\Roaming\Python\Python310\site-packages\py_mini_racer\py_mini_racer.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [ ]:
stock_zh_a_daily_one = ak.stock_zh_a_daily(symbol='sh601398', start_date='20260101', end_date='20260128', adjust="qfq")

In [6]:
stock_zh_a_daily_one.tail()

,date,open,high,low,close,volume,amount,outstanding_share,turnover
12,2026-01-21,7.53,7.56,7.34,7.35,474520755.0,3.522310e+09,2.696122e+11,0.001760
13,2026-01-22,7.35,7.38,7.25,7.27,388067394.0,2.834236e+09,2.696122e+11,0.001439
14,2026-01-23,7.27,7.29,7.17,7.21,377013409.0,2.724247e+09,2.696122e+11,0.001398
15,2026-01-26,7.19,7.32,7.15,7.24,455056572.0,3.299950e+09,2.696122e+11,0.001688
16,2026-01-27,7.24,7.37,7.21,7.23,469808053.0,3.430366e+09,2.696122e+11,0.001743


In [2]:
import akshare as ak

stock_zh_a_spot_em_df = ak.stock_zh_a_spot()
print(stock_zh_a_spot_em_df)

Please wait for a moment:   0%|          | 0/69 [00:00<?, ?it/s]

            代码    名称     最新价   涨跌额    涨跌幅      买入      卖出      昨收      今开  \
0     bj920000  安徽凤凰   20.26  0.54  2.738   20.24   20.26   19.72   19.88   
1     bj920001  纬达光电   21.15  0.83  4.085   21.15   21.16   20.32   20.43   
2     bj920002  万达轴承  114.89  3.89  3.505  114.82  114.89  111.00  111.93   
3     bj920003  中诚咨询   33.58  0.29  0.871   33.58   33.59   33.29   33.33   
4     bj920005  鼎佳精密   42.78  0.80  1.906   42.78   42.79   41.98   42.01   
...        ...   ...     ...   ...    ...     ...     ...     ...     ...   
5464  sz301667   纳百川   78.13 -3.26 -4.005   78.12   78.13   81.39   79.00   
5465  sz301668  昊创瑞通   50.98  0.55  1.091   50.98   50.99   50.43   50.45   
5466  sz301678   新恒汇   71.83  1.90  2.717   71.82   71.83   69.93   70.02   
5467  sz301687   新广益   64.71 -0.59 -0.904   64.71   64.72   65.30   63.99   
5468  sz302132  中航成飞   88.01  1.96  2.278   88.01   88.02   86.05   86.44   

          最高      最低         成交量           成交额       时间戳  
0      20.38   1

In [3]:
stock_zh_a_spot_em_df.head()

,代码,名称,最新价,涨跌额,涨跌幅,买入,卖出,昨收,今开,最高,最低,成交量,成交额,时间戳
0,bj920000,安徽凤凰,20.26,0.54,2.738,20.24,20.26,19.72,19.88,20.38,19.67,2223921.0,44512647.0,15:30:00
1,bj920001,纬达光电,21.15,0.83,4.085,21.15,21.16,20.32,20.43,21.26,20.08,3598715.0,74877507.0,15:30:00
2,bj920002,万达轴承,114.89,3.89,3.505,114.82,114.89,111.00,111.93,115.15,110.51,1741672.0,196564968.0,15:30:00
3,bj920003,中诚咨询,33.58,0.29,0.871,33.58,33.59,33.29,33.33,33.77,33.24,990851.0,33214123.0,15:30:00
4,bj920005,鼎佳精密,42.78,0.80,1.906,42.78,42.79,41.98,42.01,42.86,42.01,997644.0,42330540.0,15:30:00


### 获取个股信息

In [2]:
df=ak.stock_zh_a_hist(
        symbol="000933",
        period="daily",
        start_date='20250101',
        # end_date=end_date,
        adjust="",
    )

In [3]:
df.head()

,日期,股票代码,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率
0,2025-01-02,000933,16.77,16.84,17.45,16.60,372112,6.346743e+08,5.03,-0.36,-0.06,1.66
1,2025-01-03,000933,16.84,17.57,17.89,16.84,705518,1.239095e+09,6.24,4.33,0.73,3.15
2,2025-01-06,000933,17.60,17.43,18.12,17.20,570868,1.009033e+09,5.24,-0.80,-0.14,2.54
3,2025-01-07,000933,17.41,17.52,17.85,17.17,383530,6.707478e+08,3.90,0.52,0.09,1.71
4,2025-01-08,000933,17.39,16.99,17.50,16.68,438163,7.429856e+08,4.68,-3.03,-0.53,1.95


### 抽取最近一个月数据，分析下趋势线

In [45]:
df=df.sort_values('日期',ascending=False).iloc[0:30,:]
df=df.sort_values('日期')
df['t']=np.arange(len(df))

In [46]:
df.head()

,日期,股票代码,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率,t
181,2025-09-29,000933,19.50,19.78,19.81,19.09,497726,9.717307e+08,3.72,2.17,0.42,2.21,0
182,2025-09-30,000933,19.71,20.01,20.38,19.62,404858,8.113761e+08,3.84,1.16,0.23,1.80,1
183,2025-10-09,000933,20.68,21.21,21.24,20.08,790861,1.637621e+09,5.80,6.00,1.20,3.52,2
184,2025-10-10,000933,20.99,21.65,22.01,20.77,771479,1.664364e+09,5.85,2.07,0.44,3.43,3
185,2025-10-13,000933,20.80,21.28,21.39,20.59,657934,1.380835e+09,3.70,-1.71,-0.37,2.93,4


In [47]:
model = LinearRegression()
model.fit(df[['t']], df['最低'])

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [48]:
slope = model.coef_[0]
intercept = model.intercept_

print("趋势线斜率 =", slope)
print("趋势线截距 =", intercept)

# 计算整段趋势线
df['TrendLine'] = model.predict(df[['t']])

趋势线斜率 = 0.24107452725250278
趋势线截距 = 19.930086021505375


In [51]:
from pyecharts.charts import Kline, Line, Grid
from pyecharts import options as opts
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

def kline_with_low_regression(df):
    # --- 1. 准备数据 ---
    df = df.copy()
    df['t'] = np.arange(len(df))

    # 选最低价中最低的 20% 用于拟合趋势线（更接近人工画法）
    # threshold = df['Low'].quantile(0.2)
    # df_fit = df[df['Low'] <= threshold]

    # --- 2. 线性回归 ---
    model = LinearRegression()
    model.fit(df[['t']], df['最低'],sample_weight=df['成交量'])
    df['TrendLine'] = model.predict(df[['t']])

    # --- 3. 构造 K 线图 ---
    kline = (
        Kline()
        .add_xaxis(df['日期'].tolist())
        .add_yaxis(
            "K线",
            df[['开盘', '收盘', '最低', '最高']].values.tolist(),
            itemstyle_opts=opts.ItemStyleOpts(color="#ec0000", color0="#00da3c",
                                              border_color="#8A0000", border_color0="#008F28"),
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title="K线 + Low 回归趋势线"),
            datazoom_opts=[opts.DataZoomOpts(), opts.DataZoomOpts(type_="inside")],
            xaxis_opts=opts.AxisOpts(type_="category"),
            # yaxis_opts=opts.AxisOpts(scale=True),
            tooltip_opts=opts.TooltipOpts(trigger="axis")
        )
    )

    # --- 4. 趋势线 ---
    line = (
        Line()
        .add_xaxis(df['日期'].tolist())
        .add_yaxis(
            "Low 回归趋势线",
            df['TrendLine'].round(2).tolist(),
            is_smooth=True,
            linestyle_opts=opts.LineStyleOpts(width=2, color="orange"),
            label_opts=opts.LabelOpts(is_show=False),
        )
    )

    # --- 5. 叠加图层 ---
    overlap = kline.overlap(line)

    # --- 6. 使用 Grid 合成图 ---
    grid = (
        Grid()
        .add(overlap, grid_opts=opts.GridOpts())
    )
    return grid
# 假设 df 已有你的行情数据
chart = kline_with_low_regression(df)
chart.render("kline_regression.html")
# chart.render_notebook()


'c:\\Users\\cyw\\Desktop\\jupyternotebook\\git-python\\GP\\趋势分析\\kline_regression.html'

In [53]:
def kline_with_low_high_regression_and_ma(df):
    df = df.copy()

    # 时间序列
    df['t'] = np.arange(len(df))

    # ====== 1. 成交量加权趋势线（低点）======
    model_low = LinearRegression()
    model_low.fit(df[['t']], df['最低'], sample_weight=df['成交量'])
    df['TrendLow'] = model_low.predict(df[['t']])

    # ====== 2. 成交量加权趋势线（高点）======
    model_high = LinearRegression()
    model_high.fit(df[['t']], df['最高'], sample_weight=df['成交量'])
    df['TrendHigh'] = model_high.predict(df[['t']])

    # ====== 3. 均线（MA5、MA20）======
    df['MA5'] = df['收盘'].rolling(5).mean()
    df['MA20'] = df['收盘'].rolling(20).mean()

    # 次日 MA5 & MA20（shift -1）
    df['MA5_next'] = df['MA5'].shift(-1)
    df['MA20_next'] = df['MA20'].shift(-1)

    # ====== 4. K线图 ======
    kline = (
        Kline()
        .add_xaxis(df['日期'].tolist())
        .add_yaxis(
            "K线",
            df[['开盘','收盘','最低','最高']].values.tolist(),
            itemstyle_opts=opts.ItemStyleOpts(
                color="#ec0000",
                color0="#00da3c",
                border_color="#8A0000",
                border_color0="#008F28",
            ),
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title="成交量加权趋势通道 + 次日 MA5/MA20"),
            datazoom_opts=[opts.DataZoomOpts(), opts.DataZoomOpts(type_="inside")],
            xaxis_opts=opts.AxisOpts(type_="category"),
            yaxis_opts=opts.AxisOpts(),
        )
    )

    # ====== 5. 趋势线（低点 & 高点）======
    line_trend_low = (
        Line()
        .add_xaxis(df['日期'].tolist())
        .add_yaxis(
            "Trend Low (Volume Weighted)",
            df["TrendLow"].round(2).tolist(),
            is_smooth=True,
            linestyle_opts=opts.LineStyleOpts(color="#ff9900", width=2),
            label_opts=opts.LabelOpts(is_show=False),
        )
    )

    line_trend_high = (
        Line()
        .add_xaxis(df['日期'].tolist())
        .add_yaxis(
            "Trend High (Volume Weighted)",
            df["TrendHigh"].round(2).tolist(),
            is_smooth=True,
            linestyle_opts=opts.LineStyleOpts(color="#ff6600", width=2),
            label_opts=opts.LabelOpts(is_show=False),
        )
    )

    # ====== 6. 均线（MA5、MA20）======
    line_ma5 = (
        Line()
        .add_xaxis(df['日期'].tolist())
        .add_yaxis(
            "MA5",
            df["MA5"].round(2).tolist(),
            is_smooth=True,
            linestyle_opts=opts.LineStyleOpts(width=1),
            label_opts=opts.LabelOpts(is_show=False),
        )
    )
    line_ma20 = (
        Line()
        .add_xaxis(df['日期'].tolist())
        .add_yaxis(
            "MA20",
            df["MA20"].round(2).tolist(),
            is_smooth=True,
            linestyle_opts=opts.LineStyleOpts(width=1),
            label_opts=opts.LabelOpts(is_show=False),
        )
    )

    # ====== 7. 合并 ======
    overlap = (
        kline
        .overlap(line_trend_low)
        .overlap(line_trend_high)
        .overlap(line_ma5)
        .overlap(line_ma20)
    )

    # ====== 8. 输出次日 MA 值 ======
    ma_next_info = df[['日期','MA5_next','MA20_next']].dropna().tail(1)

    return overlap, ma_next_info

In [56]:
# 假设 df 已有你的行情数据
chart,ma_next_info = kline_with_low_high_regression_and_ma(df)
chart.render("kline_regression_em.html")
chart.render_notebook()


In [57]:
ma_next_info

,日期,MA5_next,MA20_next
209,2025-11-14,26.376,25.3695
